In [1]:
import time

# time.sleep(60 * 60 * 1) # sleep 1 hours

In [2]:
# SSD imports
import torch
from models.ssd300.model import SSD, ResNet
from models.ssd300.utils import generate_dboxes, Encoder
from ssd300_utils import *

# utility imports
from tqdm.notebook import tqdm, trange
import numpy as np
import os


os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
np.set_printoptions(precision=3, linewidth=1000)
torch.set_printoptions(precision=3, linewidth=1000)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Load Model

In [3]:
# load model
ssd = SSD(backbone=ResNet(), num_classes=81).to(device)
dboxes = generate_dboxes(model='ssd')
encoder = Encoder(dboxes)

checkpoint = torch.load('data/models/SSD300.pth')
ssd.load_state_dict(checkpoint["model_state_dict"])
ssd.eval();

#### I-FGSM

In [4]:
from src.utils import selective_l1_loss_indexed as selective_l1_loss
from src.utils import l1_loss


def fgsm(image: torch.tensor, model: torch.nn.Module, steps: int = 5, epsilon: float = 0.1, 
        threshold: float = 0.5, loss_fn: str = 'selective', device: str = 'cuda'):
    # prep targets
    mask = torch.zeros_like(image, requires_grad=True, device=device)
    image.requires_grad = True
    criterion = selective_l1_loss if loss_fn == 'selective' else l1_loss

    for i in range(steps):
        _, probs = model((mask + image).contiguous())
        probs = torch.nn.Softmax(dim=1)(probs)[:, 1:, :]
        # print(i, (probs>threshold).sum().sum().sum())
        loss = criterion(probs, threshold=threshold)
        model.zero_grad()
        loss.backward()
        # print((probs > threshold).to(torch.float).sum().sum().sum())

        gradient = mask.grad.data.detach()
        mask.data = mask.data - gradient.sign() * epsilon
        mask.grad.data.zero_()
        image.grad.data.zero_()

    # fix the changes
    cloacked_image = (image + mask).detach()

    return cloacked_image

In [5]:
for fname in os.listdir('data/coco_samples')[:1]:
    with torch.no_grad():
        image, _ = read_image(f'data/coco_samples/{fname}')
        _, probs = ssd(image_to_tensor(image).contiguous())
        probs = torch.nn.Softmax(dim=1)(probs)[:, 1:, :]
        print(1 - (probs > 0.05).to(torch.float).mean().item())

0.999981751825544


In [6]:
%%script echo skipping
import time

images = []
iters = 5

for fname in os.listdir('data/coco_samples')[:6]:
    image, _ = read_image(f'data/coco_samples/{fname}')
    images.append(image_to_tensor(image))
    
for threshold in [0, 0.8]:
    for steps in [1, 5]:
        ti = time.time()

        for _ in range(iters):
            for img in images:
                fgsm(img, ssd, steps=steps, threshold=threshold)
        
        method = '' if threshold == 0 else 'S-'
        attack = 'FGSM' if steps == 1 else 'PGD'
        et = (time.time() - ti) / len(images) / iters
        print(f'{method}{attack}: {et:.3f}s per img')

'''
FGSM: 0.095s per img
PGD: 0.168s per img
S-FGSM: 0.034s per img
S-PGD: 0.167s per img
'''

skipping


#### Image by Image Cloaking

In [7]:
%%script echo skipping
annotated_images = []
annotated_cloacked_images = []
differences = []

for fname in tqdm(os.listdir('data/coco_samples')[:6]):
    ''' Process vanilla image '''
    image, dims = read_image(f'data/coco_samples/{fname}', shape=(300,300))
    annotated_image = annotate_image(image, ssd, encoder, confidence=0.5)

    ''' Cloak and re-process image'''
    cloacked_image = fgsm(image_to_tensor(image), ssd, steps=10, epsilon=0.01, threshold=0.5)
    cloacked_image = image_to_numpy(cloacked_image)
    annotated_cloacked_image = annotate_image(cloacked_image, ssd, encoder, confidence=0.5)
    difference = abs(image - cloacked_image)

    annotated_images.append(annotated_image)
    annotated_cloacked_images.append(annotated_cloacked_image)
    differences.append(difference * 5)
    # save_image(f'data/coco/val_samples_ssd300/{fname}', cloacked_image, dims)


show_images_sequence(annotated_images, annotated_cloacked_images, differences, nrows=1, ncols=6)

skipping


#### Batch Cloaking

In [8]:
%%script echo skipping
''' batch version of cell above'''
annotated_images = []
annotated_cloacked_images = []
differences = []

tensor_images = []

for fname in os.listdir('data/coco_samples')[:6]:
    ''' Process vanilla image '''
    image, dims = read_image(f'data/coco_samples/{fname}', shape=(300,300))
    annotated_image = annotate_image(image, ssd, encoder, confidence=0.3)

    tensor_images.append(image_to_tensor(image))
    annotated_images.append(annotated_image)


# cloaks
''' Cloak image'''
tensor_images = torch.vstack(tensor_images)
cloacked_images = fgsm(tensor_images, ssd, steps=10, epsilon=0.01, threshold=0.5)
differences = [image_to_numpy(diff.unsqueeze(0)) for diff in abs(tensor_images.detach() - cloacked_images) * 5]


''' Re-process image '''
for cloacked_image in cloacked_images:
    cloacked_image = image_to_numpy(cloacked_image.unsqueeze(0))
    annotated_cloacked_image = annotate_image(cloacked_image, ssd, encoder, confidence=0.3)
    annotated_cloacked_images.append(annotated_cloacked_image)

show_images_sequence(annotated_images, annotated_cloacked_images, differences, nrows=1, ncols=6)

skipping


### COCO Attack

In [9]:
%%script echo skipping
from src.coco_eval import prepare_dataset_folder 

for loss_fn in ['l1', 'selective'][1:]:
    for steps in [1, 5][1:]:
        for epsilon in [1/10, 1/50][1:]:
            threshold = 0.1
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_cloaked/ssd300_{dst_folder}')


            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc = dst_folder):
                image, dims = read_image(src, shape=(300,300))
                image = image_to_tensor(image)

                # 4.4? 1? 2.3?
                cloacked_image = fgsm(image, ssd, steps, 4.4 * epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image, dims)

skipping


### COCO Evaluation - FiftyOne

In [10]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone
from ssd300_utils import detections_fiftyone_format

dataset = load_coco_dataset_in_fiftyone('data/coco', use_cached=False, max_samples=None)
# session = fo.launch_app(dataset, desktop=True)  # comment me out if running the cell multiple times

skipping


In [11]:
%%script echo skipping
for sample in tqdm(dataset):
    detections = detections_fiftyone_format(sample.filepath, ssd, encoder)
    sample['predictions'] = detections
    sample.save()

# session.view = dataset.view()
results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
print(f'mAP: {results.mAP():.3f}')

skipping


### COCO Evaluation - Full

In [12]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone
from ssd300_utils import detections_fiftyone_format
res = {}


for loss_fn in ['l1', 'selective'][1:]:
    for steps in [1, 5][1:]:
        for epsilon in [1/10, 1/50][1:]:
            method = 'fgsm' if steps == 1 else 'pgd'

            coco_folder = f'C:/Users/tcilloni/Desktop/coco_cloaked/ssd300_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False)
            
            for sample in tqdm(dataset, desc=coco_folder[46:]):
                detections = detections_fiftyone_format(sample.filepath, ssd, encoder)
                sample['predictions'] = detections
                sample.save()

            # session.view = dataset.view()
            results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')
            res[f'{loss_fn} {method} {epsilon:.3f}'] = results.mAP()

print(res)

skipping


# Pascal VOC
### Attack

In [13]:
# %%script echo skipping
from src.coco_eval import prepare_voc_dataset_folder

for loss_fn in ['l1', 'selective'][1:]:
    for steps in [1, 3]:
        for epsilon in [1/10, 1/50][:1]:
            threshold = 0.05
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_voc_dataset_folder('data/pascal', f'C:/Users/tcilloni/Desktop/pascal_cloaked/ssd300_{dst_folder}')

            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc = dst_folder):
                image, dims = read_image(src, shape=(300,300))
                image = image_to_tensor(image)

                cloacked_image = fgsm(image, ssd, steps, 4.4 * epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image, dims)

selective_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

selective_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

C:\Users\tcilloni\anaconda3\envs\loss_exploration\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [1, 3, 300, 300], strides() = [3, 1, 900, 3]
param.sizes() = [1, 3, 300, 300], strides() = [270000, 1, 900, 3] (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch/csrc/autograd/functions/accumulate_grad.h:192.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


### Evaluation

In [14]:
# %%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from ssd300_utils import detections_fiftyone_format
res = {}


for loss_fn in ['l1', 'selective'][1:]:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50][:1]:
            method = 'fgsm' if steps == 1 else 'pgd'

            voc_folder = f'C:/Users/tcilloni/Desktop/pascal_cloaked/ssd300_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_pascal_dataset_in_fiftyone(voc_folder, use_cached=False)
            
            for sample in tqdm(dataset, desc=voc_folder):
                detections = detections_fiftyone_format(sample.filepath, ssd, encoder, dataset='pascal')
                sample['predictions'] = detections
                sample.save()

            results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')
            res[f'{loss_fn} {method} {epsilon:.3f}'] = results.mAP()

print(res)

 100% |███████████████| 4952/4952 [24.7s elapsed, 0s remaining, 202.1 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/ssd300_selective_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [1.1m elapsed, 0s remaining, 73.0 samples/s]       
selective fgsm 0.100	 mAP: 0.200
 100% |███████████████| 4952/4952 [23.7s elapsed, 0s remaining, 200.7 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/ssd300_selective_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [40.8s elapsed, 0s remaining, 114.9 samples/s]      
selective pgd 0.100	 mAP: 0.048
{'selective fgsm 0.100': 0.20037981598570118, 'selective pgd 0.100': 0.04788252436837208}


### Baseline

In [15]:
%%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from ssd300_utils import *

dataset = load_pascal_dataset_in_fiftyone('data/pascal', use_cached=False)

for sample in tqdm(dataset, desc='baseline'):
    detections = detections_fiftyone_format(sample.filepath, ssd, encoder, dataset='pascal')
    sample['predictions'] = detections
    sample.save()

results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
print(f'baseline mAP: {results.mAP():.3f}')

skipping


In [16]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}